In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install libraries
!pip install transformers datasets evaluate
!conda install protobuf
!pip install accelerate -U
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.8 MB/s eta 0

In [ ]:
import pdfplumber
import re
import os
import csv

def extract_information_from_cv(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        cv_text = ""
        for page in pdf.pages:
            cv_text += page.extract_text()

    summary_pattern = re.compile(r'Summary\n(.+?)\n', re.DOTALL)
    summary_match = summary_pattern.search(cv_text)
    summary = summary_match.group(1) if summary_match else None

    experience_pattern = re.compile(r'Experience\n(.+?)\nEducation', re.DOTALL)
    experience_match = experience_pattern.search(cv_text)
    experience = experience_match.group(1) if experience_match else None

    education_pattern = re.compile(r'Education\n(.+?)\nSkills', re.DOTALL)
    education_match = education_pattern.search(cv_text)
    education = education_match.group(1) if education_match else None

    skills_pattern = re.compile(r'Skills\n(.+)', re.DOTALL)
    skills_match = skills_pattern.search(cv_text)
    skills = skills_match.group(1) if skills_match else None

    if experience is None:
        experience = "experience"
    if education is None:
        education = "education"

    data = experience + "\n" + education
    return data

In [ ]:

data = [
    ["domain", "about"],
]

filename = "/content/drive/MyDrive/Tareq/resum.csv"
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)
print(f"CSV file '{filename}' created successfully.")

CSV file '/content/drive/MyDrive/Tareq/resum.csv' created successfully.


In [ ]:
parent_path = "/content/drive/MyDrive/Tareq/data"
subdirectory = os.listdir(parent_path)
row_data = []
for dir in subdirectory:
	dir_name = dir
	print(dir_name)
	path_of_dir = os.path.join(parent_path,dir)
	list_of_cv = os.listdir(path_of_dir)
	for cv in list_of_cv:
		cv_path = os.path.join(path_of_dir,cv)
		data = extract_information_from_cv(cv_path)
		data_list = [dir_name, data]
		row_data.append(data_list)
	print(f"Finish {dir_name} domain")

BPO
Finish BPO domain
ACCOUNTANT
Finish ACCOUNTANT domain
ADVOCATE
Finish ADVOCATE domain
APPAREL
Finish APPAREL domain
AUTOMOBILE
Finish AUTOMOBILE domain
BUSINESS-DEVELOPMENT
Finish BUSINESS-DEVELOPMENT domain
AVIATION
Finish AVIATION domain
AGRICULTURE
Finish AGRICULTURE domain
ARTS
Finish ARTS domain
BANKING
Finish BANKING domain
HR
Finish HR domain
FITNESS
Finish FITNESS domain
DESIGNER
Finish DESIGNER domain
CHEF
Finish CHEF domain
FINANCE
Finish FINANCE domain
CONSULTANT
Finish CONSULTANT domain
DIGITAL-MEDIA
Finish DIGITAL-MEDIA domain
ENGINEERING
Finish ENGINEERING domain
HEALTHCARE
Finish HEALTHCARE domain
CONSTRUCTION
Finish CONSTRUCTION domain
TEACHER
Finish TEACHER domain
SALES
Finish SALES domain
PUBLIC-RELATIONS
Finish PUBLIC-RELATIONS domain
INFORMATION-TECHNOLOGY
Finish INFORMATION-TECHNOLOGY domain


In [ ]:
import csv
filename = "/content/drive/MyDrive/Tareq/resum.csv"
with open(filename, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_data)
print("New data appended to the CSV file.")

New data appended to the CSV file.


In [ ]:
# Data processing
import pandas as pd
import numpy as np
from datasets import Dataset
# Modeling
# import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

# Model performance evaluation
import evaluate

In [ ]:
# Read in data
md_data = pd.read_csv("/content/drive/MyDrive/Tareq/resum.csv",names=['labels','about'],skiprows=[0])
# Take a look at the data
df = md_data.head(len(md_data))
df.drop(index=0)
len(df)

2484

In [ ]:
# print(df['label'].value_counts())
df['labels'] = df['labels'].replace(['ACCOUNTANT',
 'ADVOCATE',
 'AGRICULTURE',
 'APPAREL',
 'ARTS',
 'AUTOMOBILE',
 'AVIATION',
 'BANKING',
 'BPO',
 'BUSINESS-DEVELOPMENT',
 'CHEF',
 'CONSTRUCTION',
 'CONSULTANT',
 'DESIGNER',
 'DIGITAL-MEDIA',
 'ENGINEERING',
 'FINANCE',
 'FITNESS',
 'HEALTHCARE',
 'HR',
 'INFORMATION-TECHNOLOGY',
 'PUBLIC-RELATIONS',
 'SALES',
 'TEACHER'],['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
)
print(df['labels'].value_counts())
df.to_csv("resume_updated.csv", index=False)

20    120
9     120
15    118
1     118
0     118
10    118
16    118
6     117
17    117
22    116
18    115
7     115
12    115
11    112
21    111
19    110
13    107
4     103
23    102
3      97
14     96
2      63
5      36
8      22
Name: labels, dtype: int64


In [ ]:
md_data = pd.read_csv("/content/resume_updated.csv",names=['labels','about'],skiprows=[0])

In [ ]:
# Training dataset
train_data = md_data.sample(frac=0.8, random_state=42)

# Testing dataset
test_data = md_data.drop(train_data.index)

# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

The training dataset has 1987 records.
The testing dataset has 497 records.


In [ ]:
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["about"],
                     truncation=True,
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

In [ ]:
# Take a look at the data
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['labels', 'about', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1987
})
Dataset({
    features: ['labels', 'about', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 497
})


In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=24)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./resume/",
    logging_dir='./resume/logs',
    logging_strategy='epoch',
    logging_steps=10,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-7,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [ ]:
# Function to compute the metric
import torch
torch.cuda.is_available()


def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,3.330500,3.282542,0.036217
2,3.293700,3.262873,0.042254
3,3.262800,3.253838,0.046278
4,3.262800,3.249619,0.046278
5,3.247500,3.247017,0.048290
6,3.248300,3.245304,0.046278
7,3.245500,3.244239,0.046278
8,3.248400,3.243854,0.046278


TrainOutput(global_step=1000, training_loss=3.267433837890625, metrics={'train_runtime': 1786.9594, 'train_samples_per_second': 8.896, 'train_steps_per_second': 0.56, 'total_flos': 4183239484440576.0, 'train_loss': 3.267433837890625, 'epoch': 8.0})

In [ ]:
# Trainer evaluate
trainer.evaluate(dataset_test)

{'eval_loss': 3.243854284286499,
 'eval_accuracy': 0.04627766599597585,
 'eval_runtime': 19.1309,
 'eval_samples_per_second': 25.979,
 'eval_steps_per_second': 1.673,
 'epoch': 8.0}

In [ ]:
# Save tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/Tareq/data/resume_transformer/')

# Save model
trainer.save_model('/content/drive/MyDrive/Tareq/data/resume_transformer/')